In [2]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('sentence-transformers/all-MiniLM-L6-v2', max_length=512)
#For Example
scores = model.predict([('How many people live in Berlin?', 'Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.'), 
                        ('How many people live in Berlin?', 'Berlin is well known for its museums.')])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
scores

array([0.5051551 , 0.50396323], dtype=float32)

In [59]:
import sqlparse

def get_column(sql_query):
    parsed_query = sqlparse.parse(sql_query)
    columns = []
    # Loop through the tokens in the parsed query
    for stmt in parsed_query:
        for token in stmt.tokens:
            if isinstance(token, sqlparse.sql.IdentifierList):
                for identifier in token.get_identifiers():
                    columns.append(identifier.get_real_name())
    return columns

In [60]:
import pandas as pd

desc_col = {}

description_df = pd.read_excel('src/New_query_Description.xlsx',header=1)
description_df = description_df[['Column','Description']]
for i, row in description_df.iterrows():
    if type(row['Column']) == float:
        continue
    desc_col[i] = row['Column']

In [61]:
pointx_cols = pd.read_csv('src/pointx_fbs_rpt_dly.csv').columns.to_list()
assert len(pointx_cols) == len(desc_col), "Length of columns in Pointx table and Descriptions are not equal"

In [63]:
compare_df = pd.read_csv('src/compare_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()

In [66]:
for sql in sql_queries[:3]:
    print(sql)
    print(get_column(sql))

SELECT event_date, COUNT(DISTINCT user_pseudo_id)
FROM (
    SELECT event_date, ga_session_id, user_pseudo_id
    FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
    GROUP BY event_date, ga_session_id, user_pseudo_id
    HAVING SUM(engagement_time_msec) > 10*1000
)
GROUP BY event_date
ORDER BY event_date ASC
['event_date', 'COUNT']
SELECT event_month, COUNT(DISTINCT user_pseudo_id)
FROM (
    SELECT event_month, ga_session_id, user_pseudo_id
    FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
    GROUP BY event_month, ga_session_id, user_pseudo_id
    HAVING SUM(engagement_time_msec) > 10*1000
)
GROUP BY event_month
ORDER BY event_month ASC
['event_month', 'COUNT']
SELECT AVG(cnt) FROM
  (SELECT event_date, COUNT(DISTINCT user_pseudo_id) as cnt
  FROM (
      SELECT event_date, ga_session_id, user_pseudo_id
      FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
      GROUP BY event_date, ga_session_id, user_pseudo_id
      HAVING SUM(engagement_time_msec) > 10*1000
  )
  GROUP BY event_

In [70]:
import sqlparse

# Your SQL query with a subquery
sql_query = """
SELECT column1, column2
FROM (
    SELECT sub_column1, sub_column2
    FROM your_table
    WHERE sub_condition
) AS subquery
WHERE condition
"""

# Parse the SQL query using sqlparse
parsed_query = sqlparse.parse(sql_query)

# Get the column names from the parsed query
columns = []

# Loop through the tokens in the parsed query
for stmt in parsed_query:
    for token in stmt.tokens:
        print(token)
        if isinstance(token, sqlparse.sql.IdentifierList):
            for identifier in token.get_identifiers():
                columns.append(identifier.get_real_name())

print(columns)




SELECT
 
column1, column2


FROM
 
(
    SELECT sub_column1, sub_column2
    FROM your_table
    WHERE sub_condition
) AS subquery


WHERE condition

['column1', 'column2']


In [68]:
parsed_query

(<Statement ' SELEC...' at 0x2A1DB9CB0>,)

In [89]:
import sqlparse
def queries(d):
  if type(d) != sqlparse.sql.Token:
     paren = isinstance(d, sqlparse.sql.Parenthesis)
     v = [queries(i) for i in (d if not paren else d[1:-1])]
     print(paren,d,v)
     subseq, qrs = ''.join(str(i[0]) for i in v), [x for _, y in v for x in y]
     if [*d][paren].value == 'SELECT':
        return '<subquery>', [subseq]+qrs
     return subseq, qrs
  return d, []

s="""SELECT name, email
     FROM (SELECT * FROM user WHERE email IS NOT NULL)
     WHERE id IN (SELECT cID FROM customer WHERE points > 5)
"""
_, subqueries = queries(sqlparse.parse(s)[0])

False name [(<Name 'name' at 0x2A1D76C20>, [])]
False email [(<Name 'email' at 0x2A1DDB4C0>, [])]
False name, email [('name', []), (<Punctuation ',' at 0x2A1D75AE0>, []), (<Whitespace ' ' at 0x2A1DB0340>, []), ('email', [])]
False email [(<Name 'email' at 0x2A1DDBCA0>, [])]
False WHERE email IS NOT NULL [(<Keyword 'WHERE' at 0x2A1DDBBE0>, []), (<Whitespace ' ' at 0x2A1DDBC40>, []), ('email', []), (<Whitespace ' ' at 0x2A1DDBD00>, []), (<Keyword 'IS' at 0x2A1DDBD60>, []), (<Whitespace ' ' at 0x2A1DDBDC0>, []), (<Keyword 'NOT NU...' at 0x2A1DDBE20>, [])]
True (SELECT * FROM user WHERE email IS NOT NULL) [(<DML 'SELECT' at 0x2A1DDB8E0>, []), (<Whitespace ' ' at 0x2A1DDB940>, []), (<Wildcard '*' at 0x2A1DDB9A0>, []), (<Whitespace ' ' at 0x2A1DDBA00>, []), (<Keyword 'FROM' at 0x2A1DDBA60>, []), (<Whitespace ' ' at 0x2A1DDBAC0>, []), (<Keyword 'user' at 0x2A1DDBB20>, []), (<Whitespace ' ' at 0x2A1DDBB80>, []), ('WHERE email IS NOT NULL', [])]
False id [(<Name 'id' at 0x2A1DE8220>, [])]
False

In [82]:
subqueries

['SELECT name, email\n     FROM <subquery>\n     WHERE id IN <subquery>\n',
 'SELECT * FROM user WHERE email IS NOT NULL',
 'SELECT cID FROM customer WHERE points > 5']

In [79]:
type(sqlparse.parse(s)[0])

sqlparse.sql.Statement